# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later
import sys
import os
import json
import pandas as pd
sys.path.insert(1, '../../../../src/')
import gt_modules.evaluation as evaluation
from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-sport7-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series ("Association Football Club explorative search") 

Consider the following exploratory information need:

> investigate the association football clubs and find the main BGPs related to this sport. Compare clubs in countries and championships won. Explore also the participation of the countries in the FIFA World Cup.

**Important consideration: in wikidata, we are talking of ASSOCIATION FOOTBALL**

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q11571`| Cristiano Ronaldo       | node |





Also consider

```
wd:Q11571 ?p ?obj .
```

is the BGP to retrieve all **properties of Cristiano Ronaldo**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for association football club

2. Identify the BGP for association football league

3. Return for each north american country the number of association football clubs

4. Identify the BGP for the UEFA Champions League.

5. Consider the countries which have at least two different clubs of their nationality who won the UEFA Champions League. Return the list of these countries with the number of different clubs who won the UEFA Champions League (the result set must be triples of country IRI, label and #clubs).

6. Return for each european country the top level league (the result set must be a list of country IRI and label, and league IRI and label).

7. Identify the BGP of the FIFA World Cup

8. Consider countries who participated at the FIFA World Cup more than 10 times. Return these countries with the number of apparitions (the result set must be triples of country IRI, label and #apparitions).

9. Consider the countries that won the FIFA World Cup at least twice. Find the association football clubs of those countries won the UEFA Champions League (the result set must be couples of clubs IRI and label).


In [2]:
## startup the evaluation
# setup the file and create the empty json
ipname = "s7.ipynb"
pt = os.getcwd()+os.sep+ipname
evaluation.setup(pt)

The index of this workflow is: 0_0


## Task 1 
Identify the BGP for football club

In [2]:
# starting from Cristiano Ronaldo I look at his properties
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
    wd:Q11571 ?pr ?obj.
    ?pr sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('pr', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('pr', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('pr', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('pr', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('pr', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('pr', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[

In [3]:
# starting from Cristiano Ronaldo I look at his occupations
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    wd:Q11571 wdt:P54 ?obj.
    ?obj <http://schema.org/name> ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q3590758'), ('name', 'Portugal Olympic football team')]
[('obj', 'http://www.wikidata.org/entity/Q1422'), ('name', 'Juventus F.C.')]
[('obj', 'http://www.wikidata.org/entity/Q18656'), ('name', 'Manchester United F.C.')]
[('obj', 'http://www.wikidata.org/entity/Q8682'), ('name', 'Real Madrid CF')]
[('obj', 'http://www.wikidata.org/entity/Q1630430'), ('name', 'Portugal national under-21 football team')]
[('obj', 'http://www.wikidata.org/entity/Q267245'), ('name', 'Portugal national association football team')]
[('obj', 'http://www.wikidata.org/entity/Q75729'), ('name', 'Sporting CP')]
[('obj', 'http://www.wikidata.org/entity/Q1772776'), ('name', 'Portugal national under-20 football team')]
[('obj', 'http://www.wikidata.org/entity/Q3590754'), ('name', 'Portugal national under-17 football team')]
[('obj', 'http://www.wikidata.org/entity/Q21079208'), ('name', 'Portugal national under-15 football team')]
10


In [22]:
# look for the istance of the results found above
queryString = """
SELECT DISTINCT ?inst ?name
WHERE { 
    wd:Q11571 wdt:P54 ?team .
    ?team wdt:P31 ?inst.  
    ?inst <http://schema.org/name> ?name .
}
"""

print("Results")
x= run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q23759293'), ('name', 'national Olympic football team')]
[('inst', 'http://www.wikidata.org/entity/Q15944511'), ('name', 'association football team')]
[('inst', 'http://www.wikidata.org/entity/Q476028'), ('name', 'association football club')]
[('inst', 'http://www.wikidata.org/entity/Q4830453'), ('name', 'business')]
[('inst', 'http://www.wikidata.org/entity/Q891723'), ('name', 'public company')]
[('inst', 'http://www.wikidata.org/entity/Q6979593'), ('name', 'national association football team')]
[('inst', 'http://www.wikidata.org/entity/Q847017'), ('name', 'sports club')]
[('inst', 'http://www.wikidata.org/entity/Q23847779'), ('name', 'national under-23 association football team')]
[('inst', 'http://www.wikidata.org/entity/Q23904673'), ('name', 'national under-15 association football team')]
9


In [3]:
### insert the result of TASK 1 in the file
og_uri = "http://www.wikidata.org/entity/Q476028"
og_name = "association football club"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"1", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 0_0
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_0.json
JSON object updated


`wd:Q476028` is the node of the association football club in wikidata

## Task 2
Identify the BGP for football league

In [4]:
# In the query above I found Real Madrid, wd:Q8682 look at its properties
queryString = """
SELECT DISTINCT ?p ?pName
WHERE { 

wd:Q8682 ?p ?obj.

?p <http://schema.org/name> ?pName .
}
"""

print("Results")
x = run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('pName', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('pName', 'founded by')]
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('pName', 'home venue')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('pName', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('pName', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('pName', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pName', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pName', 'headquarters location')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award 

In [5]:
# Found property league (wdt:P118). Let's see the result
queryString = """
SELECT DISTINCT ?league ?name
WHERE { 

wd:Q8682 wdt:P118 ?league.

?league <http://schema.org/name> ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('league', 'http://www.wikidata.org/entity/Q324867'), ('name', 'La Liga')]
1


In [6]:
# Real Madrid plays in La Liga. Find the instance.
queryString = """
SELECT DISTINCT ?inst ?name
WHERE { 

wd:Q8682 wdt:P118 ?league.
?league wdt:P31 ?inst.

?inst <http://schema.org/name> ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q15991290'), ('name', 'professional sports league')]
[('inst', 'http://www.wikidata.org/entity/Q15991303'), ('name', 'association football league')]
2


In [4]:
### insert the result of TASK 2 in the file
og_uri = "http://www.wikidata.org/entity/Q15991303"
og_name = "association football league"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"2", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 0_0
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_0.json
JSON object updated


`wd:Q15991303` is the node of the association football league in wikidata

## Task 3
Return for each north american country the number of association football clubs

First of all we have to identify countries of North America. 

In [7]:
# bind the football player and look at his nationality
queryString = """
SELECT DISTINCT ?nation ?name
WHERE { 

?player wdt:P106 wd:Q937857;
        wdt:P27 ?nation.

?nation sc:name ?name .
}
ORDER BY (?name)
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('nation', 'http://www.wikidata.org/entity/Q13417403'), ('name', '.الجزائر')]
[('nation', 'http://www.wikidata.org/entity/Q889'), ('name', 'Afghanistan')]
[('nation', 'http://www.wikidata.org/entity/Q170366'), ('name', 'Ahwaz')]
[('nation', 'http://www.wikidata.org/entity/Q137014'), ('name', 'AlSaudiah')]
[('nation', 'http://www.wikidata.org/entity/Q222'), ('name', 'Albania')]
[('nation', 'http://www.wikidata.org/entity/Q1242155'), ('name', 'Albanians in Kosovo')]
[('nation', 'http://www.wikidata.org/entity/Q262'), ('name', 'Algeria')]
[('nation', 'http://www.wikidata.org/entity/Q218272'), ('name', 'Algeria')]
[('nation', 'http://www.wikidata.org/entity/Q2415901'), ('name', 'Allied-occupied Germany')]
[('nation', 'http://www.wikidata.org/entity/Q16641'), ('name', 'American Samoa')]
[('nation', 'http://www.wikidata.org/entity/Q228'), ('name', 'Andorra')]
[('nation', 'http://www.wikidata.org/entity/Q916'), ('name', 'Angola')]
[('nation', 'http://www.wikidata.org/entity/Q25228'),

In [8]:
# look at properties of the nationalities
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 

?player wdt:P106 wd:Q937857;
        wdt:P27 ?nation.
?nation wdt:P31 ?x.

?x sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q1489259'), ('name', 'superpower')]
[('x', 'http://www.wikidata.org/entity/Q1323642'), ('name', 'transcontinental country')]
[('x', 'http://www.wikidata.org/entity/Q99541706'), ('name', 'historical unrecognized state')]
[('x', 'http://www.wikidata.org/entity/Q112099'), ('name', 'island nation')]
[('x', 'http://www.wikidata.org/entity/Q123480'), ('name', 'landlocked country')]
[('x', 'http://www.wikidata.org/entity/Q1520223'), ('name', 'constitutional republic')]
[('x', 'http://www.wikidata.org/entity/Q179164'), ('name', 'unitary state')]
[('x', 'http://www.wikidata.org/entity/Q20181813'), ('name', 'colonial power')]
[('x', 'http://www.wikidata.org/entity/Q3024240'), ('name', 'historical country')]
[('x', 'http://www.wikidata.org/entity/Q3624078'), ('name', 'sovereign state')]
[('x', 'http://www.wikidata.org/entity/Q41614'), ('name', 'constitutional monarchy')]
[('x', 'http://www.wikidata.org/entity/Q4209223'), ('name', 'Rechtsstaat')]
[('x

In [9]:
# Q6256 is the class of countries
# look at properties to find the properties continent
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 

?nation wdt:P31 wd:Q6256;
        ?prop ?x.

?prop sc:name ?name .
FILTER(REGEX(?name,"ontin")).
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P30'), ('name', 'continent')]
1


In [57]:
# Q6256 is the class of countries
# P30 is the properties continent: list them and find Europe
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 

?nation wdt:P31 wd:Q6256;
        wdt:P30 ?x.

?x sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q15'), ('name', 'Africa')]
[('x', 'http://www.wikidata.org/entity/Q18'), ('name', 'South America')]
[('x', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
[('x', 'http://www.wikidata.org/entity/Q48'), ('name', 'Asia')]
[('x', 'http://www.wikidata.org/entity/Q49'), ('name', 'North America')]
[('x', 'http://www.wikidata.org/entity/Q538'), ('name', 'Insular Oceania')]
[('x', 'http://www.wikidata.org/entity/Q5401'), ('name', 'Eurasia')]
[('x', 'http://www.wikidata.org/entity/Q27611'), ('name', 'Central America')]
8


`wd:Q49` is the North America node.

In [10]:
# look at properties of intances of football league to find something related with countries
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
?league wdt:P31 wd:Q15991303;
        ?prop ?country.
?country wdt:P31 wd:Q6256.

?prop sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('prop', 'http://www.wikidata.org/prop/direct/P2541'), ('name', 'operating area')]
[('prop', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
3


In [6]:
# using all the informations found above I can find how many leagues each country have
queryString = """
SELECT DISTINCT ?country ?name COUNT(DISTINCT ?league) AS ?leagues
WHERE { 
#leagues
?league wdt:P31 wd:Q15991303;
        wdt:P17 ?country.
# north american countries
?country wdt:P31 wd:Q6256;
        wdt:P30 wd:Q49.

?country sc:name ?name .
}
GROUP BY ?country ?name
ORDER BY ?name
"""

print("Results")
x = run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q781'), ('name', 'Antigua and Barbuda'), ('leagues', '1')]
[('country', 'http://www.wikidata.org/entity/Q244'), ('name', 'Barbados'), ('leagues', '1')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('name', 'Canada'), ('leagues', '16')]
[('country', 'http://www.wikidata.org/entity/Q800'), ('name', 'Costa Rica'), ('leagues', '3')]
[('country', 'http://www.wikidata.org/entity/Q241'), ('name', 'Cuba'), ('leagues', '1')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('name', 'Denmark'), ('leagues', '8')]
[('country', 'http://www.wikidata.org/entity/Q784'), ('name', 'Dominica'), ('leagues', '1')]
[('country', 'http://www.wikidata.org/entity/Q786'), ('name', 'Dominican Republic'), ('leagues', '2')]
[('country', 'http://www.wikidata.org/entity/Q792'), ('name', 'El Salvador'), ('leagues', '2')]
[('country', 'http://www.wikidata.org/entity/Q769'), ('name', 'Grenada'), ('leagues', '1')]
[('country', 'http://www.wikidata.org/entity/

In [7]:
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    leagues = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= leagues
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"3", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 0_0
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_0.json
JSON object updated


## Task 4
Identify BGP for the UEFA Champions League

In [14]:
# football players properties
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 

?team wdt:P31 wd:Q476028;
        ?prop ?obj.
FILTER(!isLiteral(?obj)).
?prop sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P115'), ('name', 'home venue')]
[('prop', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('prop', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('prop', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('prop', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('prop', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('prop', 'http://www.wikidata.org/prop/direct/P1454'), ('name', 'legal form')]
[('prop', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('prop', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location')]
[('prop', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('prop', 'http://www.wikidata.org/prop/direct/P1830'), ('name', 'owner of')]
[('pr

In [16]:
# football players properties
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 

?team wdt:P31 wd:Q476028;
        wdt:P2522 ?obj.
?obj sc:name ?name .
filter(regex(?name,"UEFA")).
}
"""

print("Results")
x = run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q756120'), ('name', '1986–87 UEFA Cup')]
[('obj', 'http://www.wikidata.org/entity/Q833923'), ('name', '1981–82 UEFA Cup')]
[('obj', 'http://www.wikidata.org/entity/Q18128921'), ('name', '2016 UEFA Super Cup')]
[('obj', 'http://www.wikidata.org/entity/Q30608'), ('name', '2013–14 UEFA Champions League')]
[('obj', 'http://www.wikidata.org/entity/Q13416183'), ('name', '2015–16 UEFA Champions League')]
[('obj', 'http://www.wikidata.org/entity/Q308996'), ('name', '2004–05 UEFA Champions League')]
[('obj', 'http://www.wikidata.org/entity/Q18740961'), ('name', '2016–17 UEFA Champions League')]
[('obj', 'http://www.wikidata.org/entity/Q20972117'), ('name', '2017–18 UEFA Champions League')]
[('obj', 'http://www.wikidata.org/entity/Q28842926'), ('name', '2019 UEFA Champions League Final')]
[('obj', 'http://www.wikidata.org/entity/Q754140'), ('name', '1985–86 UEFA Cup')]
[('obj', 'http://www.wikidata.org/entity/Q756140'), ('name', '1997–98 UEFA Cup'

In [18]:
# see the instance of one of the UEFA CL edition
queryString = """
SELECT DISTINCT ?inst ?name
WHERE { 

wd:Q330050 wdt:P31 ?inst.
?inst sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q27020041'), ('name', 'sports season')]
1


In [19]:
# see the instance of one of the UEFA CL edition
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 

wd:Q330050 ?pr ?inst.
?pr sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('pr', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('pr', 'http://www.wikidata.org/prop/direct/P1350'), ('name', 'number of matches played/races/starts')]
[('pr', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('pr', 'http://www.wikidata.org/prop/direct/P3450'), ('name', 'sports season of league or competition')]
[('pr', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('pr', 'http://www.wikidata.org/prop/direct/P393'), ('name', 'edition number')]
[('pr', 'http://www.wikidata.org/prop/direct/P580'), ('name', 'start time')]
[('pr', 'http://www.wikidata.org/prop/direct/P582'), ('name', 'end time')]
[('pr', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('pr', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P664'), ('name', 'organizer')

In [20]:
# see the instance of one of the UEFA CL edition
queryString = """
SELECT DISTINCT ?inst ?name
WHERE { 

wd:Q330050 wdt:P3450 ?inst.
?inst sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q18756'), ('name', 'UEFA Champions League')]
1


In [8]:
### insert the result of TASK 4 in the file
og_uri = "http://www.wikidata.org/entity/Q18756"
og_name = "UEFA Champions League"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"4", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 0_0
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_0.json
JSON object updated


`wd:Q18756` is the UEFA Champions League

## Task 5
Consider the countries which have at least two different clubs of their nationality who won the UEFA Champions League. Return the list of these countries with the number of different clubs who won the UEFA Champions League (the result set must be triples of country IRI, label and #clubs).

In [22]:
# see the instance of one of the UEFA CL edition
queryString = """
SELECT DISTINCT ?inst ?name
WHERE { 

?inst wdt:P3450 wd:Q18756.
?inst sc:name ?name .
}
ORDER BY ?name
"""

print("Results")
x = run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q219384'), ('name', '1955–56 European Cup')]
[('inst', 'http://www.wikidata.org/entity/Q245202'), ('name', '1956–57 European Cup')]
[('inst', 'http://www.wikidata.org/entity/Q309318'), ('name', '1957–58 European Cup')]
[('inst', 'http://www.wikidata.org/entity/Q309316'), ('name', '1958–59 European Cup')]
[('inst', 'http://www.wikidata.org/entity/Q392535'), ('name', '1959–60 European Cup')]
[('inst', 'http://www.wikidata.org/entity/Q330686'), ('name', '1960–61 European Cup')]
[('inst', 'http://www.wikidata.org/entity/Q497028'), ('name', '1961–62 European Cup')]
[('inst', 'http://www.wikidata.org/entity/Q500074'), ('name', '1962–63 European Cup')]
[('inst', 'http://www.wikidata.org/entity/Q500084'), ('name', '1963–64 European Cup')]
[('inst', 'http://www.wikidata.org/entity/Q497052'), ('name', '1964–65 European Cup')]
[('inst', 'http://www.wikidata.org/entity/Q573225'), ('name', '1965–66 European Cup')]
[('inst', 'http://www.wikidata.org/

In [23]:
# see the instance of one of the UEFA CL edition
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 

?inst wdt:P3450 wd:Q18756;
    ?pr ?x.
?pr sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('pr', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('pr', 'http://www.wikidata.org/prop/direct/P1350'), ('name', 'number of matches played/races/starts')]
[('pr', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('pr', 'http://www.wikidata.org/prop/direct/P1705'), ('name', 'native label')]
[('pr', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('pr', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class')]
[('pr', 'http://www.wikidata.org/prop/direct/P2348'), ('name', 'time period')]
[('pr', 'http://www.wikidata.org/prop/direct/P2500'), ('name', 'league level below')]
[('pr', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]


In [25]:
# see the instance of one of the UEFA CL edition
queryString = """
SELECT ?x ?club COUNT(*) AS ?victories
WHERE { 

?inst wdt:P3450 wd:Q18756;
    wdt:P1346 ?x.
?x sc:name ?club .
}
GROUP BY  ?x ?club
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q1422'), ('club', 'Juventus F.C.'), ('victories', '2')]
[('x', 'http://www.wikidata.org/entity/Q131499'), ('club', 'S.L. Benfica'), ('victories', '2')]
[('x', 'http://www.wikidata.org/entity/Q18656'), ('club', 'Manchester United F.C.'), ('victories', '4')]
[('x', 'http://www.wikidata.org/entity/Q15789'), ('club', 'FC Bayern Munich'), ('victories', '6')]
[('x', 'http://www.wikidata.org/entity/Q1130849'), ('club', 'Liverpool F.C.'), ('victories', '6')]
[('x', 'http://www.wikidata.org/entity/Q128446'), ('club', 'F.C. Porto'), ('victories', '2')]
[('x', 'http://www.wikidata.org/entity/Q51974'), ('club', 'Hamburger SV'), ('victories', '1')]
[('x', 'http://www.wikidata.org/entity/Q81888'), ('club', 'AFC Ajax'), ('victories', '4')]
[('x', 'http://www.wikidata.org/entity/Q179658'), ('club', 'FCSB'), ('victories', '1')]
[('x', 'http://www.wikidata.org/entity/Q19490'), ('club', 'Nottingham Forest F.C.'), ('victories', '2')]
[('x', 'http://www.wikida

In [9]:
# see the instance of one of the UEFA CL edition
queryString = """
SELECT ?c ?country COUNT(DISTINCT ?club) AS ?club
WHERE { 

?inst wdt:P3450 wd:Q18756;
    wdt:P1346 ?club.
?club wdt:P17 ?c.
?c sc:name ?country .
}
GROUP BY ?c ?country
HAVING (COUNT(DISTINCT ?club) > 1)
"""

print("Results")
x = run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q45'), ('country', 'Portugal'), ('club', '2')]
[('c', 'http://www.wikidata.org/entity/Q38'), ('country', 'Italy'), ('club', '3')]
[('c', 'http://www.wikidata.org/entity/Q55'), ('country', 'Netherlands'), ('club', '3')]
[('c', 'http://www.wikidata.org/entity/Q183'), ('country', 'Germany'), ('club', '3')]
[('c', 'http://www.wikidata.org/entity/Q29'), ('country', 'Spain'), ('club', '2')]
[('c', 'http://www.wikidata.org/entity/Q145'), ('country', 'United Kingdom'), ('club', '6')]
6


In [10]:
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    clubs = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= clubs
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"5", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 0_0
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_0.json
JSON object updated


## Task 6
Return for each european country the top level league

In [29]:
# Real Madrid plays in La Liga. Find the instance.
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 

?league wdt:P31 wd:Q15991303;
    ?pr ?x.

?pr sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P6118'), ('name', 'season ends')]
[('pr', 'http://www.wikidata.org/prop/direct/P7441'), ('name', 'WorldFootball.net competition ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P4003'), ('name', 'Facebook page ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('pr', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('pr', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('pr', 'http://www.wikidata.org/prop/direct/P1350'), ('name', 'number of matches played/races/starts')]
[('pr', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('pr', 'http://www.wikidata.org/prop/direct/P137'), ('name', 'operator')]
[('pr', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('pr', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('pr', 'http://www.wikidat

In [31]:
# Real Madrid plays in La Liga. Find the instance.
queryString = """
SELECT DISTINCT ?league ?name ?x
WHERE { 

?league wdt:P31 wd:Q15991303;
    wdt:P3983 ?x.

?league sc:name ?name .
}
LIMIT 10
"""

print("Results")
x = run_query(queryString)

Results
[('league', 'http://www.wikidata.org/entity/Q2950655'), ('name', 'Ligue 2'), ('x', '2')]
[('league', 'http://www.wikidata.org/entity/Q2792041'), ('name', '3. divisjon'), ('x', '4')]
[('league', 'http://www.wikidata.org/entity/Q162533'), ('name', 'Russian Football National League Competition'), ('x', '2')]
[('league', 'http://www.wikidata.org/entity/Q238113'), ('name', 'Slovak Super Liga'), ('x', '1')]
[('league', 'http://www.wikidata.org/entity/Q475887'), ('name', 'Bremen-Liga'), ('x', '5')]
[('league', 'http://www.wikidata.org/entity/Q555836'), ('name', 'Regionalliga West'), ('x', '4')]
[('league', 'http://www.wikidata.org/entity/Q739479'), ('name', 'Faroe Islands Premier League'), ('x', '1')]
[('league', 'http://www.wikidata.org/entity/Q247788'), ('name', 'Yemeni League'), ('x', '1')]
[('league', 'http://www.wikidata.org/entity/Q1763335'), ('name', 'Kutonen'), ('x', '7')]
[('league', 'http://www.wikidata.org/entity/Q2188121'), ('name', 'Regionalliga'), ('x', '2')]
10


In [12]:
# Real Madrid plays in La Liga. Find the instance.
queryString = """
SELECT DISTINCT ?country ?cName ?league ?name
WHERE { 

?league wdt:P31 wd:Q15991303;
    wdt:P3983 ?x;
    wdt:P17 ?country.
?country wdt:P30 wd:Q46.
FILTER(?x = 1).

?league sc:name ?name .
?country sc:name ?cName .
}
"""

print("Results")
x = run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('cName', 'Italy'), ('league', 'http://www.wikidata.org/entity/Q15804'), ('name', 'Serie A')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('cName', 'Spain'), ('league', 'http://www.wikidata.org/entity/Q324867'), ('name', 'La Liga')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('cName', 'Russia'), ('league', 'http://www.wikidata.org/entity/Q182165'), ('name', 'Russian Premier League')]
[('country', 'http://www.wikidata.org/entity/Q215'), ('cName', 'Slovenia'), ('league', 'http://www.wikidata.org/entity/Q275005'), ('name', 'Slovenian PrvaLiga')]
[('country', 'http://www.wikidata.org/entity/Q16957'), ('cName', 'German Democratic Republic'), ('league', 'http://www.wikidata.org/entity/Q157553'), ('name', 'DDR-Oberliga')]
[('country', 'http://www.wikidata.org/entity/Q37'), ('cName', 'Lithuania'), ('league', 'http://www.wikidata.org/entity/Q275665'), ('name', 'A Lyga')]
[('country', 'http://www.wikidata.org/entity/Q7

In [13]:
# add the elements to the JSON gt
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    c_uri = i[2][1]
    c_name = i[3][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = ["uri","name"]
    obj["uri"]= c_uri
    obj["name"]= c_name
    objs.append(obj)

evaluation.add_result(evaluation.get_index_workflow(pt),"6", evaluation.TYPE_REFERRED ,["uri","name"], objs)

The index of this workflow is: 0_0
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_0.json
JSON object updated


## Task 7
Identify the BGP of the FIFA World Cup

In [41]:
# print properties of the players that are not literals
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 

?player wdt:P106 wd:Q937857;
        ?prop ?x.
FILTER(!isLiteral(?x)).
?prop sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('prop', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('prop', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('prop', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('prop', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('prop', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('prop', 'http://www.wikidata.org/prop/direct/P140'), ('name', 'religion')]
[('prop', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('prop', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]
[('prop', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('prop', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different fro

In [42]:
# print objects of the properties related with partecipant in (P1344)
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 

?player wdt:P106 wd:Q937857;
        wdt:P1344 ?x.
?x sc:name ?name .
}
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q104439579'), ('name', '2020 Belarusian Premier League relegation play-offs')]
[('x', 'http://www.wikidata.org/entity/Q77927099'), ('name', '2020 Belarusian Premier League')]
[('x', 'http://www.wikidata.org/entity/Q11831154'), ('name', '2019 Africa Cup of Nations')]
[('x', 'http://www.wikidata.org/entity/Q11865693'), ('name', 'football at the 2016 Summer Olympics')]
[('x', 'http://www.wikidata.org/entity/Q8150'), ('name', '1936 Summer Olympics')]
[('x', 'http://www.wikidata.org/entity/Q48432'), ('name', '1930 FIFA World Cup')]
[('x', 'http://www.wikidata.org/entity/Q18479838'), ('name', "football at the 2016 Summer Olympics – women's tournament")]
[('x', 'http://www.wikidata.org/entity/Q62589205'), ('name', "football at the 2020 Summer Olympics – women's tournament")]
[('x', 'http://www.wikidata.org/entity/Q102920'), ('name', 'UEFA Euro 2004')]
[('x', 'http://www.wikidata.org/entity/Q1185431'), ('name', '2015 Pan American Games')]
[('x', '

I found a lot of objects that seems to be part of a super class or something else. Starting point: `wd:Q37285`

In [43]:
# print objects of the properties related with partecipant in (P1344)
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 

wd:Q37285 ?prop ?x.
?prop sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('prop', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('prop', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('prop', 'http://www.wikidata.org/prop/direct/P1350'), ('name', 'number of matches played/races/starts')]
[('prop', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('prop', 'http://www.wikidata.org/prop/direct/P1546'), ('name', 'motto')]
[('prop', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('prop', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class')]
[('prop', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'L

In [44]:
# use the property sports league or competition
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 

wd:Q37285 wdt:P3450 ?x.
?x sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q19317'), ('name', 'FIFA World Cup')]
1


In [14]:
### insert the result of TASK 7 in the file
og_uri = "http://www.wikidata.org/entity/Q19317"
og_name = "FIFA World Cup"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"7", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 0_0
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_0.json
JSON object updated


`wd:Q19317` is the FIFA World Cup 

## Task 8
Consider countries who participated at the FIFA World Cup at least 10 times. Return these countries with the number of apparitions (the result set must be triples of country IRI, label and #apparitions).

In [50]:
# print properties of the players that are not literals
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 

?wc wdt:P3450 wd:Q19317.
?x ?pr ?wc.
?pr sc:name ?name
}
"""

print("Results")
x = run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1423'), ('name', 'template has topic')]
[('pr', 'http://www.wikidata.org/prop/direct/P1268'), ('name', 'represents')]
[('pr', 'http://www.wikidata.org/prop/direct/P6291'), ('name', 'advertises')]
[('pr', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('pr', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('pr', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('pr', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('pr', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('pr', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('pr', 'http://www.wikidata.org/prop/direct/P179'), ('name', 'part of the series')]
[('pr', 'http://www.wikidata.org/prop/direct/P180'), ('name', 'depicts')]
[('pr', 'http://www.wikidata.org/prop/direct/P2348'), ('name', 'time period')]
[('pr', 'http://www.wikidata.org/prop/direct/P2522

In [42]:
# print properties of the players that are not literals
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 

?wc wdt:P3450 wd:Q19317.
?wc ?pr ?x.
?pr sc:name ?name
}
"""

print("Results")
x = run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P4967'), ('name', 'prerequisite')]
[('pr', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('pr', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('pr', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('pr', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('pr', 'http://www.wikidata.org/prop/direct/P1350'), ('name', 'number of matches played/races/starts')]
[('pr', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('pr', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('pr', 'http://www.wikidata.org/prop/direct/P1546'), ('name', 'motto')]
[('pr', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('pr', 'http://www.wikidata.org/prop/direct/P1617'), ('name', 'BBC Things ID')]
[('pr', 'http://www.wikidata.org/prop

In [45]:
# print properties of the players that are not literals
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 

?wc wdt:P3450 wd:Q19317.
?wc wdt:P1346 ?x.
?x sc:name ?name.

}
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q676899'), ('name', 'Italy national association football team')]
[('x', 'http://www.wikidata.org/entity/Q134916'), ('name', 'Uruguay national football team')]
[('x', 'http://www.wikidata.org/entity/Q47762'), ('name', 'England national association football team')]
[('x', 'http://www.wikidata.org/entity/Q47774'), ('name', 'France national association football team')]
[('x', 'http://www.wikidata.org/entity/Q42267'), ('name', 'Spain national association football team')]
[('x', 'http://www.wikidata.org/entity/Q79800'), ('name', 'Argentina national football team')]
[('x', 'http://www.wikidata.org/entity/Q83459'), ('name', 'Brazil national football team')]
[('x', 'http://www.wikidata.org/entity/Q43310'), ('name', 'Germany national association football team')]
8


In [46]:
# print properties of the players that are not literals
queryString = """
SELECT DISTINCT ?inst ?name
WHERE { 

?wc wdt:P3450 wd:Q19317.
?wc wdt:P1346 ?x.
?x wdt:P31 ?inst.
?inst sc:name ?name.

}
"""

print("Results")
x = run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q6979593'), ('name', 'national association football team')]
1


In [54]:
# print properties of the players that are not literals
queryString = """
SELECT ?country ?name COUNT(*) AS ?app
WHERE { 

?wc wdt:P3450 wd:Q19317.
?x wdt:P1344 ?wc.
?x wdt:P17 ?country;
    wdt:P31 wd:Q6979593.
?country sc:name ?name.

}
GROUP BY ?country ?name
HAVING (COUNT(*) > 10)
"""

print("Results")
x = run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy'), ('app', '18')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany'), ('app', '18')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil'), ('app', '20')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('name', 'Belgium'), ('app', '12')]
[('country', 'http://www.wikidata.org/entity/Q34'), ('name', 'Sweden'), ('app', '12')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('name', 'Spain'), ('app', '14')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('name', 'Argentina'), ('app', '15')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('app', '14')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom'), ('app', '26')]
[('country', 'http://www.wikidata.org/entity/Q77'), ('name', 'Uruguay'), ('app', '12')]
[('country', 'http://www.wikidata.org/entity/Q96'), ('name', 'Mexico'), ('app', '15')]
11


## Task 9
Consider the countries that won the FIFA World Cup at least twice. Find the association football clubs of those countries won the UEFA Champions League.

In [87]:
# print the countries with the number of FIFA World Cup won
queryString = """
SELECT ?country ?cName (COUNT(?wc) AS ?wcs)
WHERE { 

?wc wdt:P3450 wd:Q19317;
    wdt:P1346 ?x.
    
?country wdt:P31 wd:Q6256.
?x wdt:P17 ?country.
    
?country sc:name ?cName.
}
GROUP BY ?country ?cName

"""

print("Results")
x = run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('cName', 'Italy'), ('wcs', '4')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('cName', 'Brazil'), ('wcs', '5')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('cName', 'Argentina'), ('wcs', '2')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('cName', 'France'), ('wcs', '2')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('cName', 'United Kingdom'), ('wcs', '1')]
[('country', 'http://www.wikidata.org/entity/Q77'), ('cName', 'Uruguay'), ('wcs', '2')]
6


In [55]:
# print the countries with the number of FIFA World Cup won
queryString = """
SELECT DISTINCT ?club ?name
WHERE{
    {
        SELECT ?country 
        WHERE { 
            ?wc wdt:P3450 wd:Q19317;
                wdt:P1346 ?x.

            ?country wdt:P31 wd:Q6256.
            ?x wdt:P17 ?country.
        }
        GROUP BY ?country
        HAVING (COUNT(DISTINCT ?wc) > 1)
    }
    
    ?inst wdt:P3450 wd:Q18756;
        wdt:P1346 ?club.
    ?club wdt:P17 ?country.
    
    ?club sc:name ?name.
}

"""

print("Results")
x = run_query(queryString)

Results
[('club', 'http://www.wikidata.org/entity/Q132885'), ('name', 'Olympique de Marseille')]
[('club', 'http://www.wikidata.org/entity/Q1422'), ('name', 'Juventus F.C.')]
[('club', 'http://www.wikidata.org/entity/Q1543'), ('name', 'A.C. Milan')]
[('club', 'http://www.wikidata.org/entity/Q631'), ('name', 'FC Internazionale Milano')]
4
